# Introduction to Star/Galaxy Separation

By Melissa Graham.

Last verified to run Fri Mar 25 2022, with Weekly 2021_49.

## Set Up
import some packages, start TAP service (see, e.g., NB 01, 02, 06)

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
font = {'size'   : 18}
plt.rc('font', **font)

from lsst.rsp import get_tap_service, retrieve_query
service = get_tap_service()

## Get Objects

define the TAP query to get a bunch of objects

In [ ]:
query = "SELECT obj.objectId, obj.mag_g, obj.mag_r, obj.mag_i, obj.extendedness, "\
        "truth.match_objectId, truth.truth_type "\
        "FROM dp01_dc2_catalogs.object as obj "\
        "JOIN dp01_dc2_catalogs.truth_match as truth "\
        "ON truth.match_objectId = obj.objectId "\
        "WHERE CONTAINS(POINT('ICRS', obj.ra, obj.dec), "\
        "CIRCLE('ICRS', 62.0, -37.0, 0.50)) = 1 "\
        "AND truth.match_objectid >= 0 "\
        "AND truth.is_good_match = 1"
print(query)

In [ ]:
%%time
results = service.search(query)
print('Query returned %s matched objects.' % len(results))

In [ ]:
results.fieldnames

## Explore Extendedness Parameter

how many true stars are extended? how many galaxies are NOT extended?

In [ ]:
str_index = np.where(results['truth_type'] == 2)
gal_index = np.where(results['truth_type'] == 1)

In [ ]:
fig, ax = plt.subplots( 2, figsize=(14,7), sharex=True )

ax[0].hist( results['extendedness'][str_index], bins=20, histtype='step', \
           color='darkgreen', label='stars' )
ax[0].legend(loc='upper right')

ax[1].hist( results['extendedness'][gal_index], bins=20, histtype='step', \
           color='darkorange', label='galaxies' )
ax[1].legend(loc='upper left')

ax[1].set_xlabel('extendedness')

ax[0].set_ylabel('# stars')
ax[1].set_ylabel('# galaxies')

plt.show()

what's up with the extended stars and point-like galaxies? are they all faint?

In [ ]:
ext_str_index = np.where( (results['truth_type'] == 2) & (results['extendedness'] == 1) )
pnt_str_index = np.where( (results['truth_type'] == 2) & (results['extendedness'] == 0) )

ext_gal_index = np.where( (results['truth_type'] == 1) & (results['extendedness'] == 1) )
pnt_gal_index = np.where( (results['truth_type'] == 1) & (results['extendedness'] == 0) )

In [ ]:
fig, ax = plt.subplots( 2, figsize=(14,7), sharex=True )

ax[0].hist( results['mag_i'][ext_str_index], bins=30, histtype='step', \
           color='limegreen', lw=2, label='extended stars' )
ax[0].hist( results['mag_i'][pnt_str_index], bins=30, histtype='step', \
           color='darkgreen', label='point-like stars' )
ax[0].legend(loc='upper left')

ax[1].hist( results['mag_i'][ext_gal_index], bins=30, histtype='step', \
           color='darkorange', label='extended galaxies' )
ax[1].hist( results['mag_i'][pnt_gal_index], bins=30, histtype='step', \
           color='magenta', lw=2, label='point-like galaxies' )
ax[1].legend(loc='upper left')

ax[1].set_xlabel('apparent i-band magnitude')

ax[0].set_ylabel('# stars')
ax[1].set_ylabel('# galaxies')
ax[1].set_xlim([15,30])

plt.show()